In [1]:
import pandas as pd
import numpy as np
import requests
import bs4
import json
import os

In [2]:
from shapely.geometry import Point
import geopandas as gpd   

# Importar Modulo existente

## Importando el modulo de geo_calcs

In [3]:
import geo_calculations as geo

# Cargando data

## Centros Culturales

In [4]:
centros_culturales = requests.get('https://datos.madrid.es/egob/catalogo/200304-0-centros-culturales.json')
centros_culturales = centros_culturales.json()
df_centros_culturales = pd.json_normalize(centros_culturales['@graph'])
df_centros_culturales = pd.DataFrame(df_centros_culturales[["organization.organization-name" , "address.street-address" , "location.latitude" , "location.longitude"]])
df_centros_culturales = df_centros_culturales.assign(Tipo_Centro = "Centros culturales")

## Museos

In [5]:
museos = requests.get('https://datos.madrid.es/egob/catalogo/201132-0-museos.json')
museos = museos.json()
df_museos = pd.json_normalize(museos['@graph'])
df_museos = pd.DataFrame(df_museos[["organization.organization-name" , "address.street-address" , "location.latitude" , "location.longitude"]])
df_museos = df_museos.assign(Tipo_Centro = "Museos")
#df_museos["coordinadas_mercator"] = df_museos.apply(lambda x : to_mercator(x["location.latitude"],x["location.longitude"]),axis=1)
#df_museos

## Concatenando y calculando las coordenadas en mercator

In [6]:
df_museo_cc = pd.concat([df_centros_culturales , df_museos])
df_museo_cc["coordinadas_mercator"] = df_museo_cc.apply(lambda x : geo.to_mercator(x["location.latitude"],x["location.longitude"]),axis=1)

Ejemplo de las primeras 5 entradas..

In [10]:
df_museo_cc.head()

,organization.organization-name,address.street-address,location.latitude,location.longitude,Tipo_Centro,coordinadas_mercator
0,Auditorio Carmen Laforet (Ciudad Lineal),CALLE JAZMIN 46,40.479608,-3.665799,Centros culturales,POINT (4506169.379345033 -408353.5729058877)
1,Auditorio al aire libre. Parque del Calero,CALLE JOSE DEL HIERRO 3,40.436300,-3.650410,Centros culturales,POINT (4501348.276905228 -406636.9344770753)
2,Auditorio y sala de exposiciones Paco de Lucía...,AVENIDA LAS AGUILAS 2 A,40.385132,-3.761912,Centros culturales,POINT (4495652.310977291 -419075.2884951149)
3,Centro Comunitario Casino de la Reina,CALLE CASINO 3,40.406793,-3.704609,Centros culturales,POINT (4498063.582233449 -412682.7875456674)
4,Centro Cultural - Centro Socio Cultural Villa ...,CALLE BOTICA 7 y 10,40.473402,-3.578523,Centros culturales,POINT (4505478.547905593 -398618.6384765698)


In [13]:
#Aca simplemente cambie el path
df_bicimadstations = pd.read_json("./data/bicimadstations.json")
#aca tenian un bug grave, estaban asugnando longitud a latitud y viceversa
lon =[float(index["geometry_coordinates"].split(",")[0].replace("[", "")) for row, index in df_bicimadstations.iterrows()]
lat = [float(index["geometry_coordinates"].split(",")[1].replace("]","")) for row, index in df_bicimadstations.iterrows()]
df_bicimadstations["latitud"] = lat
df_bicimadstations["longitud"] = lon
df_bicimadstations = pd.DataFrame(df_bicimadstations[["name" , "address" , "latitud" , "longitud"]])
#notar que he utilizado la funcion del modulo `geo_calculations`, el cual importe como "geo"
df_bicimadstations["coordinadas_mercator"] = df_bicimadstations.apply(lambda x : geo.to_mercator(x["latitud"],x["longitud"]),axis=1)

# Distancia para un solo lugar

In [14]:
def bici_mas_cercana_alternative(data_lugar, df_bicis):
    """
    Args:
        data_lugar: un diccionario/objecto con el museo/centro
        df_bicis: un dataframe con la data de bicimadrid
    """
    df_bicis["distancia_total"] = df_bicis.apply(lambda x : x['coordinadas_mercator'].distance(data_lugar['coordinadas_mercator']),axis=1)
    return df_bicis.sort_values(by = "distancia_total", ascending = True).iloc[0]

## Probando

In [18]:
i = str(input("Introduzca lugar"))
a = df_museo_cc[df_museo_cc["organization.organization-name"] == i].iloc[0]
bici_mas_cercana_alternative(a, df_bicimadstations.copy())

Introduzca lugar Auditorio al aire libre. Parque del Calero


name                                                 C.D. Concepción
address                                   Calle Jose del Hierro nº 5
latitud                                                    40.437151
longitud                                                   -3.648359
coordinadas_mercator    POINT (4501443.047273415 -406408.1503400214)
distancia_total                                           247.636031
Name: 231, dtype: object

,organization.organization-name,address.street-address,location.latitude,location.longitude,Tipo_Centro,coordinadas_mercator,bici_name,bici_address,bici_lat,bici_lon,bici_coordinadas_mercator,bici_dist_total
1,Auditorio al aire libre. Parque del Calero,CALLE JOSE DEL HIERRO 3,40.4363,-3.65041,Centros culturales,POINT (4501443.047273415 -406408.1503400214),C.D. Concepción,Calle Jose del Hierro nº 5,40.437151,-3.648359,POINT (4501443.047273415 -406408.1503400214),247.636031


## Otra prueba, correr para un centro aleatorio

In [15]:
bici_mas_cercana_alternative(df_museo_cc.sample(n=1).iloc[0], df_bicimadstations.copy())

name                                                         Legazpi
address                                           Calle Bolívar nº 3
latitud                                                    40.391472
longitud                                                   -3.694194
coordinadas_mercator    POINT (4496358.117694666 -411521.0631924719)
distancia_total                                           185.821094
Name: 179, dtype: object

# Calculando Para todas las organizaciones a la vez

In [16]:
df_museo_cc[['bici_name','bici_address','bici_lat','bici_lon','bici_coordinadas_mercator','bici_dist_total']] = df_museo_cc.apply(lambda x: bici_mas_cercana_alternative(x, df_bicimadstations.copy()), axis=1)

In [17]:
df_museo_cc

,organization.organization-name,address.street-address,location.latitude,location.longitude,Tipo_Centro,coordinadas_mercator,bici_name,bici_address,bici_lat,bici_lon,bici_coordinadas_mercator,bici_dist_total
0,Auditorio Carmen Laforet (Ciudad Lineal),CALLE JAZMIN 46,40.479608,-3.665799,Centros culturales,POINT (4506169.379345033 -408353.5729058877),Manuel Caldeiro,Paseo de la Castellana nº 298,40.479380,-3.685320,POINT (4506143.969227422 -410531.117482332),2177.692829
1,Auditorio al aire libre. Parque del Calero,CALLE JOSE DEL HIERRO 3,40.436300,-3.650410,Centros culturales,POINT (4501348.276905228 -406636.9344770753),C.D. Concepción,Calle Jose del Hierro nº 5,40.437151,-3.648359,POINT (4501443.047273415 -406408.1503400214),247.636031
2,Auditorio y sala de exposiciones Paco de Lucía...,AVENIDA LAS AGUILAS 2 A,40.385132,-3.761912,Centros culturales,POINT (4495652.310977291 -419075.2884951149),Caramuel,Calle Caramuel nº 51,40.409136,-3.728534,POINT (4498324.476616229 -415351.7319152072),4583.158606
3,Centro Comunitario Casino de la Reina,CALLE CASINO 3,40.406793,-3.704609,Centros culturales,POINT (4498063.582233449 -412682.7875456674),Embajadores 2,Glorieta de Embajadores nº 2,40.405611,-3.702259,POINT (4497932.008315246 -412420.6948701236),293.264840
4,Centro Cultural - Centro Socio Cultural Villa ...,CALLE BOTICA 7 y 10,40.473402,-3.578523,Centros culturales,POINT (4505478.547905593 -398618.6384765698),Gutierre de Cetina,Calle Gutierre de Cetina nº 77,40.429706,-3.640994,POINT (4500614.284841758 -405586.667644808),8497.910663
...,...,...,...,...,...,...,...,...,...,...,...,...
64,Real Monasterio de las Descalzas Reales,PLAZA DESCALZAS 3,40.418233,-3.706213,Museos,POINT (4499337.1563866 -412861.7774830329),Plaza de Celenque A,Plaza de Celenque nº 1,40.417370,-3.705779,POINT (4499241.080999179 -412813.3597196624),107.586058
65,Templo de Debod,CALLE FERRAZ 1,40.424251,-3.716532,Museos,POINT (4500007.033180737 -414012.8663162367),Ferraz,Calle Evaristo San Miguel nº 2,40.425394,-3.717045,POINT (4500134.319725253 -414070.0873872822),139.556854
66,The Robot Museum (Museo del Robot),CALLE SERRANO 61 PLANTA BAJA,40.432215,-3.686955,Museos,POINT (4500893.569362991 -410713.5180322552),Castellana 42,Paseo de la Castellana nº 42,40.433409,-3.687915,POINT (4501026.467520318 -410820.6351983234),170.692728
67,Yacimiento paleontológico. Estación de Metro C...,CALLE VIA CARPETANA 141,40.392541,-3.741303,Museos,POINT (4496477.095493491 -416776.1850890592),Caramuel,Calle Caramuel nº 51,40.409136,-3.728534,POINT (4498324.476616229 -415351.7319152072),2332.784572
